## Gripe

In [81]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pickle
import os


In [82]:
def carregar_dados():
    dados_2021 = pd.read_csv('dados/gripe_2021.csv', sep=';', encoding='ISO-8859-1')
    dados_2022 = pd.read_csv('dados/gripe_2022.csv', sep=';', encoding='ISO-8859-1')
    dados_2023 = pd.read_csv('dados/gripe_2023.csv', sep=';', encoding='ISO-8859-1')
    dados = pd.concat([dados_2021, dados_2022, dados_2023])
    return dados

In [ ]:
def carregar_dados_municipios():
    # Ajuste o caminho do arquivo conforme necessário.
    caminho_arquivo = 'dados/cod_municipios.csv'
    dados_municipios = pd.read_csv(caminho_arquivo, sep=';', encoding='ISO-8859-1')
    # Remova colunas e linhas desnecessárias se existirem.
    dados_municipios = dados_municipios[['UF', 'Nome_UF', 'Nome_Mesorregião', 'Nome_Microrregião', 'Código Município Completo', 'Nome_Município']]
    return dados_municipios

In [83]:
def tratar_dados(dados):
    dados['DT_NOTIFIC'] = pd.to_datetime(dados['DT_NOTIFIC'], format='%d/%m/%Y')
    dados['DT_NASC'] = pd.to_datetime(dados['DT_NASC'], format='%d/%m/%Y', errors='coerce')
    dados['ANO'] = dados['DT_NOTIFIC'].dt.year
    dados['MES'] = dados['DT_NOTIFIC'].dt.month
    dados = pd.get_dummies(dados, columns=['SG_UF_NOT', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA'])
    dados = dados.dropna(subset=['CO_REGIONA', 'DT_NASC'])
    return dados

In [84]:
def preparar_previsores_alvo(dados):
    dados_agregados = dados.groupby(['ANO', 'MES', 'CO_REGIONA']).size().reset_index(name='CASOS')
    previsores = dados_agregados.drop('CASOS', axis=1)
    alvo = dados_agregados['CASOS']
    return previsores, alvo

In [85]:
def escalonar_previsores(previsores):
    scaler = StandardScaler()
    previsores_escalados = scaler.fit_transform(previsores)
    return previsores_escalados

In [86]:
def dividir_treino_teste(previsores, alvo):
    X_train, X_test, y_train, y_test = train_test_split(previsores, alvo, test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

In [87]:
def treinar_modelos(X_train, y_train):
    modelos = {
        'KNN': KNeighborsClassifier(),
        'MLP': MLPClassifier(max_iter=50) 
    }

    for nome, modelo in modelos.items():
        modelo.fit(X_train, y_train)
        # Salvando cada modelo
        modelo_arquivo = f'./modelos/{nome}.pickle'
        with open(modelo_arquivo, 'wb') as f:
            pickle.dump(modelo, f)
    
    return modelos


In [88]:
def criar_modelo_hibrido(X_train, y_train):
    # Carregar modelos treinados
    modelos = treinar_modelos(X_train, y_train)
    modelos_carregados = {nome: pickle.load(open(f'./modelos/{nome}.pickle', 'rb')) for nome in modelos}

    voting_clf = VotingClassifier(estimators=[(nome, modelo) for nome, modelo in modelos_carregados.items()], voting='soft')
    voting_clf.fit(X_train, y_train)

    # Salvando o modelo híbrido
    modelo_hibrido_arquivo = './modelos/ModeloHibrido.pickle'
    with open(modelo_hibrido_arquivo, 'wb') as f:
        pickle.dump(voting_clf, f)
    
    return voting_clf


In [89]:
def avaliar_modelos(X_test, y_test, modelos):
    for nome, modelo_arquivo in modelos.items():
        modelo = pickle.load(open(f'./modelos/{nome}.pickle', 'rb'))
        predicoes = modelo.predict(X_test)
        acuracia = accuracy_score(y_test, predicoes)
        f1 = f1_score(y_test, predicoes, average='macro')
        print(f'{nome} - Acurácia: {acuracia}, F1 Score: {f1}')


In [90]:
def interface_interativa(dados):
    ano_widget = widgets.IntSlider(value=2024, min=dados['ANO'].min(), max=dados['ANO'].max(), step=1, description='Ano:')
    mes_widget = widgets.Dropdown(options=list(range(1, 13)), value=2, description='Mês:')
    regiao_widget = widgets.Dropdown(options=sorted(dados['CO_REGIONA'].unique()), description='Região:')
    modelo_widget = widgets.Dropdown(options=os.listdir('./modelos'), description='Modelo:')
    
    display(ano_widget, mes_widget, regiao_widget, modelo_widget)

    def realizar_predicoes(b):
        modelo_selecionado = modelo_widget.value
        caminho_modelo = os.path.join('./modelos', modelo_selecionado)
        modelo = pickle.load(open(caminho_modelo, 'rb'))
        
        entrada_predicao = pd.DataFrame({'ANO': [ano_widget.value], 'MES': [mes_widget.value], 'CO_REGIONA': [regiao_widget.value]})
        predicao = modelo.predict(entrada_predicao)
        
        with output_area:
            output_area.clear_output()
            print(f"Predição para a região {regiao_widget.value} no mês {mes_widget.value} de {ano_widget.value}: {predicao[0]}")
    
    predicao_botao = widgets.Button(description="Realizar Predição")
    output_area = widgets.Output()
    predicao_botao.on_click(realizar_predicoes)
    display(predicao_botao, output_area)

In [91]:
dados = carregar_dados()
dados_tratados = tratar_dados(dados)
previsores, alvo = preparar_previsores_alvo(dados_tratados)
previsores_escalados = escalonar_previsores(previsores)
X_train, X_test, y_train, y_test = dividir_treino_teste(previsores_escalados, alvo)
modelos = treinar_modelos(X_train, y_train)
modelo_hibrido = criar_modelo_hibrido(X_train, y_train)
avaliar_modelos(X_test, y_test, modelos)
interface_interativa(dados_tratados)

/var/folders/z3/2x5_0s3j31zdf3c9g85v3rbc0000gn/T/ipykernel_21832/1863773529.py:2: DtypeWarning: Columns (15,59,61,62,63,91,93,105,107,114,116,117,118,122,143) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_2021 = pd.read_csv('dados/gripe_2021.csv', sep=';', encoding='ISO-8859-1')
/var/folders/z3/2x5_0s3j31zdf3c9g85v3rbc0000gn/T/ipykernel_21832/1863773529.py:3: DtypeWarning: Columns (15,53,59,61,62,63,91,93,120,122,143,171) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_2022 = pd.read_csv('dados/gripe_2022.csv', sep=';', encoding='ISO-8859-1')
/var/folders/z3/2x5_0s3j31zdf3c9g85v3rbc0000gn/T/ipykernel_21832/1863773529.py:4: DtypeWarning: Columns (15,18,20,23,53,59,61,63,72,74,79,91,93,109,130,175,178) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_2023 = pd.read_csv('dados/gripe_2023.csv', sep=';', encoding='ISO-8859-1')
/var/folders/z3/2x5_0s3j31zdf3c9g85v3rbc0000gn/T/ipykernel_2183

KNN - Acurácia: 0.036303630363036306, F1 Score: 0.0009935482511446954
MLP - Acurácia: 0.04026402640264026, F1 Score: 0.00031725885206889017


IntSlider(value=2024, description='Ano:', max=2024, min=2021)

Dropdown(description='Mês:', index=1, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=2)

Dropdown(description='Região:', options=(1331.0, 1332.0, 1333.0, 1334.0, 1335.0, 1336.0, 1337.0, 1338.0, 1339.…

Dropdown(description='Modelo:', options=('ModeloHibrido.pickle', 'RandomForest.pickle', 'KNN.pickle', 'MLP.pic…

Button(description='Realizar Predição', style=ButtonStyle())

Output()

/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
